In [ ]:
# ====== RADIAL TOKENIZER TEST ======
import torch
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from radial_tokenizer import RadialTokenizer  # Import from same folder

# ==== CONFIG ====
IMG_PATH = "C:/Users/denni/Downloads/test.png"
IMG_NAME = "sample1"
OUTPUT_DIR = "../output"
TOK192_DIR = os.path.join(OUTPUT_DIR, "tokens_192D")
WEIGHTS_DIR = os.path.join(OUTPUT_DIR, "projection_weights")
RING_IMG_PATH = os.path.join(OUTPUT_DIR, f"{IMG_NAME}_rings.png")

os.makedirs(TOK192_DIR, exist_ok=True)
os.makedirs(WEIGHTS_DIR, exist_ok=True)

# ==== LOAD IMAGE ====
image = cv2.imread(IMG_PATH)
if image is None:
    raise FileNotFoundError(f"Image not found at {IMG_PATH}")

image = cv2.resize(image, (128, 128))
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_norm = image_rgb / 255.0
img_tensor = torch.from_numpy(image_norm).permute(2, 0, 1).unsqueeze(0).float()

print(f"✅ Loaded Image Tensor Shape: {img_tensor.shape}")

# ==== INIT TOKENIZER ====
tokenizer = RadialTokenizer()

# ==== VISUALIZE RINGS (different colors) ====
ring_colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 165, 0)]  # Blue, Green, Red, Orange
rings = tokenizer.rings
center = tokenizer.center
image_with_rings = image_rgb.copy()

for idx, (r0, r1) in enumerate(rings):
    cv2.circle(image_with_rings, center, r1, ring_colors[idx], 1)
    if r0 > 0:
        cv2.circle(image_with_rings, center, r0, ring_colors[idx], 1)

plt.figure(figsize=(5, 5))
plt.imshow(image_with_rings)
plt.title("Radial Tokenizer Rings Overlay (Colored)")
plt.axis("off")
plt.show()

# ==== RUN TOKENIZER ====
tokens_192d = tokenizer(img_tensor)  # Shape: [1, 4, 192]
print(f"✅ Tokens Shape (192D): {tokens_192d.shape}")

# ==== SAVE OUTPUTS ====
torch.save(tokens_192d.cpu(), os.path.join(TOK192_DIR, f"{IMG_NAME}.pt"))
torch.save(tokenizer.projector.state_dict(), os.path.join(WEIGHTS_DIR, f"{IMG_NAME}.pt"))

print(f"💾 Saved 192D tokens to: {os.path.join(TOK192_DIR, f'{IMG_NAME}.pt')}")
print(f"💾 Saved projection weights to: {os.path.join(WEIGHTS_DIR, f'{IMG_NAME}.pt')}")

In [ ]:
# ====== RADIAL POSITIONAL ENCODING TEST ======
import torch
import os
from radial_positional_encoding import RadialPositionEmbedding

IMG_NAME = "sample1"
OUTPUT_DIR = "../output"
TOK192_DIR = os.path.join(OUTPUT_DIR, "tokens_192D")
ENCODED_DIR = os.path.join(OUTPUT_DIR, "encoded_tokens")

os.makedirs(ENCODED_DIR, exist_ok=True)

# ==== LOAD PROJECTED TOKENS ====
tokens_192d = torch.load(os.path.join(TOK192_DIR, f"{IMG_NAME}.pt"))

# ==== RUN POSITIONAL ENCODER ====
encoder = RadialPositionEmbedding()
encoded_tokens = encoder(tokens_192d)
torch.save(encoded_tokens.cpu(), os.path.join(ENCODED_DIR, f"encoded_{IMG_NAME}.pt"))

print(f"✅ Encoded tokens shape: {encoded_tokens.shape}")
print(f"💾 Saved encoded tokens to: {os.path.join(ENCODED_DIR, f'encoded_{IMG_NAME}.pt')}")

In [ ]:
# ====== VALIDATE OUTPUT SHAPES & CONTENT ======
import torch
import pandas as pd
import matplotlib.pyplot as plt
import os

IMG_NAME = "sample1"
OUTPUT_DIR = "../output"
TOK192_DIR = os.path.join(OUTPUT_DIR, "tokens_192D")
ENCODED_DIR = os.path.join(OUTPUT_DIR, "encoded_tokens")
WEIGHTS_DIR = os.path.join(OUTPUT_DIR, "projection_weights")

# ==== LOAD FILES ====
tokens_192d = torch.load(os.path.join(TOK192_DIR, f"{IMG_NAME}.pt"))
projection_weights = torch.load(os.path.join(WEIGHTS_DIR, f"{IMG_NAME}.pt"))
encoded_tokens = torch.load(os.path.join(ENCODED_DIR, f"encoded_{IMG_NAME}.pt"))

# ==== SHAPE & DTYPE VALIDATION ====
assert tokens_192d.shape == (1, 4, 192), f"❌ 192D token shape mismatch: {tokens_192d.shape}"
assert encoded_tokens.shape == (1, 4, 192), f"❌ Encoded token shape mismatch: {encoded_tokens.shape}"
assert tokens_192d.dtype == torch.float32, "❌ 192D tokens dtype is not float32"
assert encoded_tokens.dtype == torch.float32, "❌ Encoded tokens dtype is not float32"
print("✅ Shape and dtype checks passed.")

# ==== PRINT FULL CONTENT ====
print("\n=== FULL 192D TOKENS (Projected) ===")
df_proj = pd.DataFrame(tokens_192d.squeeze(0).detach().numpy())
print(df_proj.to_string(index=True))

print("\n=== FULL ENCODED TOKENS ===")
df_enc = pd.DataFrame(encoded_tokens.squeeze(0).detach().numpy())
print(df_enc.to_string(index=True))

print("\n=== PROJECTION WEIGHTS ===")
for name, weights in projection_weights.items():
    print(f"\nLayer: {name}")
    print(weights.detach().numpy())

# ==== VISUALIZE GRAPH: PROJECTED TOKENS ====
tokens_proj_np = tokens_192d.squeeze(0).detach().numpy()
plt.figure(figsize=(10, 4))
for i in range(4):
    plt.plot(tokens_proj_np[i], label=f"Ring {i+1}")
plt.title("192D Projected Tokens Per Ring")
plt.xlabel("Dimension")
plt.ylabel("Value")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# ==== VISUALIZE GRAPH: ENCODED TOKENS ====
tokens_enc_np = encoded_tokens.squeeze(0).detach().numpy()
plt.figure(figsize=(10, 4))
for i in range(4):
    plt.plot(tokens_enc_np[i], label=f"Ring {i+1}")
plt.title("192D Encoded Tokens Per Ring")
plt.xlabel("Dimension")
plt.ylabel("Value")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()